# 課題2(Optional) wikipediaリンクで遊ぼう


## PageRank実装してみる - ランダムサンプリング編

### 準備
データを全部使うとRAMが死んでしまうので，適宜サンプリングしたデータのみを使用してメモリを節約する．

In [0]:
from google.colab import auth, drive, files, output
drive.mount('/content/drive')

In [0]:
import os
os.chdir("/content/drive/My Drive/STEP2020")

In [0]:
size = 10000 #サンプリングするページ数

In [0]:
#pagesの読み込み
f1 = open('txt/wikipedia/pages.txt')
pages = f1.read()
f1.close()
n_pages = pages.split("\n")
pages = [n_pages[i].split("\t")[1] for i in range(len(n_pages)-1)] #最後に空行が入っていたので．
del n_pages[:]

In [0]:
#randomにindexをサンプリング
import random
indexes = [i for i in range(1483277)]
new_indexes = list(sorted(random.sample(indexes, size))) #ここを変えるとページ数が変わる．
del indexes[:]
indexes = new_indexes

In [0]:
#サンプリングしたページとindexの対応を記録
selected_pages = []
page2num = dict()
old2new = dict()
for i in range(len(indexes)):
    selected_pages.append(pages[indexes[i]])
    page2num[pages[indexes[i]]] = i
    old2new[str(indexes[i])] = i
del pages[:]
del indexes[:]
pages = selected_pages

In [0]:
#リンクの読み込み
f1 = open('txt/wikipedia/links.txt')
links = f1.read()
f1.close()
n_links = links.split("\n")
links = [tuple(map(int, n_links[i].split('\t'))) for i in range(len(n_links)-1)] #最後に空行が入っていたので．
del n_links[:]

In [8]:
len(links)

52973671

In [0]:
#必要なリンク以外捨てる
new_links = []
for i in range(len(links)):
    if (old2new.get(str(links[i][0]))) and (old2new.get(str(links[i][1]))): #inで検索するとO(N)の計算時間なので，辞書検索でO(1)にする．
        new_links.append(tuple([old2new[str(links[i][0])], old2new[str(links[i][1])]]))
del links[:]
links = new_links
#links[:5] #確認用

In [0]:
len(links)

2193

In [10]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        2.5G        9.6G        908K        605M         11G
Swap:            0B          0B          0B


### PageRank

In [0]:
import time
import numpy as np

In [12]:
#リンク行列の定義
cpu_start = time.time()
p_matrix = np.zeros((size, size))
e_matrix = np.ones((size, size))
PageRank = 100 * np.ones(size)

#遷移確率の計算
for i in range(len(links)):
    p_matrix[links[i][0]][links[i][1]] += 1
for i in range(size):
    if sum(p_matrix[i]) != 0:
        p_matrix[i] = p_matrix[i]/sum(p_matrix[i])
google_matrix = 0.85 * p_matrix + 0.15 * (1/size) * e_matrix
#ここからべき乗法の計算
for i in range(20):
    PageRank = np.dot(PageRank, google_matrix)
cpu_end = time.time()

print('実行時間：', cpu_end - cpu_start)

実行時間： 24.12190294265747


In [0]:
rank_index = list(reversed(sorted([[PageRank[i], i] for i in range(size)])))

In [16]:
#重要度高そうなページを10個出してみる．
#ランダムサンプリングによりリンク構造を破壊しているので，なかなか不思議な結果
for i in range(10):
    print(pages[rank_index[i][1]], rank_index[i][0])

市 212.1222839589447
守山市 99.46077171931489
中核市 99.46077171931489
一宮 56.5197266444788
安房神社 55.20380916940767
鉄道車両の座席 48.67007124427454
燃料電池 44.75292997086897
京都ハイジャンプ 36.954592793585
日野自動車 34.350943158535294
リラキシン 30.66264340855253
